In [65]:
import pandas as pd
from linearmodels.panel import PanelOLS
from linearmodels.iv import AbsorbingLS

In [66]:
def print_summarized_results(results):
    # UNCOMMENT BELOW TO ONLY SEE COEFFS, STD ERRORS, AND P VALUES
    
#     # Extract parameter estimates, standard errors and p-values
#     params = results.params
#     std_errors = results.std_errors.round(5)  # Round the standard errors to 5 decimal places
#     pvalues = results.pvalues.round(5)

#     # Construct a DataFrame
#     results_df = pd.DataFrame({
#         'Parameter Estimates': params,
#         'Standard Errors': std_errors,
#         'P-values': pvalues
#     })

#     # Print the resulting DataFrame
#     print(results_df)
    print(results)

# School FE

### Preliminary No Interactions Regression Function

In [67]:
def run_preliminary_regression(outcome_var, weight):
    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:].set_index(['schoolcode', 'year'])

    outcome_var = outcome_var
    exog_vars = ['schoolmode', 'black', 'hispanic', 'lowincome', 'white']

    model = PanelOLS(dependent = data[outcome_var], exog = data.loc[:, exog_vars], 
                     weights = data[weight], entity_effects = True, time_effects = True)

    panel_results_no_per = model.fit(cov_type = 'clustered', clusters = data['districtcode'])

    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:].set_index(['schoolcode', 'year'])

    outcome_var = outcome_var
    exog_vars = ['virtualper', 'hybridper', 'black', 'hispanic', 'lowincome', 'white']

    model = PanelOLS(dependent = data[outcome_var], exog = data.loc[:, exog_vars], 
                     weights = data[weight], entity_effects = True, time_effects = True)
    
    panel_results_per = model.fit(cov_type = 'clustered', clusters = data['districtcode'])

    return [panel_results_no_per, panel_results_per]

#### MATH

In [68]:
results = run_preliminary_regression('mathpass', 'totaltested')

In [69]:
MATH_NOINT_NOPER = results[0]
print_summarized_results(MATH_NOINT_NOPER)

                          PanelOLS Estimation Summary                           
Dep. Variable:               mathpass   R-squared:                        0.0323
Estimator:                   PanelOLS   R-squared (Between):             -0.7190
No. Observations:               24734   R-squared (Within):              -0.0678
Date:                Sat, Sep 23 2023   R-squared (Overall):             -0.6955
Time:                        09:26:20   Log-likelihood                -8.718e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      136.04
Entities:                        4348   P-value                           0.0000
Avg Obs:                       5.6886   Distribution:                 F(5,20376)
Min Obs:                       4.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             22.509
                            

In [70]:
MATH_NOINT_PER = results[1]
print_summarized_results(MATH_NOINT_PER)

                          PanelOLS Estimation Summary                           
Dep. Variable:               mathpass   R-squared:                        0.0453
Estimator:                   PanelOLS   R-squared (Between):             -0.6614
No. Observations:               24734   R-squared (Within):              -0.0913
Date:                Sat, Sep 23 2023   R-squared (Overall):             -0.6408
Time:                        09:26:20   Log-likelihood                -8.702e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      161.12
Entities:                        4348   P-value                           0.0000
Avg Obs:                       5.6886   Distribution:                 F(6,20375)
Min Obs:                       4.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             28.516
                            

#### ELA

In [71]:
results = run_preliminary_regression('elapass', 'totaltested')

In [72]:
ELA_NOINT_NOPER = results[0]
print_summarized_results(ELA_NOINT_NOPER)

                          PanelOLS Estimation Summary                           
Dep. Variable:                elapass   R-squared:                        0.0562
Estimator:                   PanelOLS   R-squared (Between):             -0.7668
No. Observations:               18554   R-squared (Within):               0.0106
Date:                Sat, Sep 23 2023   R-squared (Overall):             -0.7581
Time:                        09:26:21   Log-likelihood                -5.778e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      181.18
Entities:                        3318   P-value                           0.0000
Avg Obs:                       5.5919   Distribution:                 F(5,15226)
Min Obs:                       4.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             15.869
                            

In [73]:
ELA_NOINT_PER = results[1]
print_summarized_results(ELA_NOINT_PER)

                          PanelOLS Estimation Summary                           
Dep. Variable:                elapass   R-squared:                        0.0675
Estimator:                   PanelOLS   R-squared (Between):             -0.7819
No. Observations:               18554   R-squared (Within):               0.0122
Date:                Sat, Sep 23 2023   R-squared (Overall):             -0.7730
Time:                        09:26:21   Log-likelihood                -5.767e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      183.67
Entities:                        3318   P-value                           0.0000
Avg Obs:                       5.5919   Distribution:                 F(6,15225)
Min Obs:                       4.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             20.807
                            

#### DROPOUT

In [74]:
results = run_preliminary_regression('dropout', 'totalenrolled')

# School FE + State-Year Interaction

### School FE + State-Year Regression Function

In [75]:
def run_state_year_regression(outcome_var, weight):
    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:]
    data['state_year'] = data['state'] + data['year'].astype('str')

    effect = 'state_year'
    outcome_var = outcome_var
    exog_vars = ['schoolmode', 'black', 'hispanic', 'lowincome', 'white']

    absorb = data[['schoolcode', 'year', effect]].astype('category')
    model = AbsorbingLS(dependent=data[outcome_var], exog=data[exog_vars], absorb=absorb, weights=data[weight])
    panel_results_no_per = model.fit(cov_type='clustered', clusters=data['schoolcode'])
    
    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:]
    data['state_year'] = data['state'] + data['year'].astype('str')

    effect = 'state_year'
    outcome_var = outcome_var
    exog_vars = ['virtualper', 'hybridper','black', 'hispanic', 'lowincome', 'white']

    absorb = data[['schoolcode', 'year', effect]].astype('category')
    model = AbsorbingLS(dependent=data[outcome_var], exog=data[exog_vars], absorb=absorb, weights=data[weight])
    panel_results_per = model.fit(cov_type='clustered', clusters=data['schoolcode'])

    return [panel_results_no_per, panel_results_per]

#### MATH

In [76]:
results = run_state_year_regression('mathpass', 'totaltested')

In [77]:
MATH_STATEYEAR__NOPER = results[0]
print_summarized_results(MATH_STATEYEAR__NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9182
Estimator:               Absorbing LS   Adj. R-squared:                     0.9007
No. Observations:               24734   F-statistic:                        291.53
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:22   Distribution:                      chi2(5)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0319
                                        Varaibles Absorbed:                 4368.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode    -8.9480     0.9053    -9.8838     0.0000     -10.722     -7.1736
black         -0.467

In [78]:
MATH_STATEYEAR__NOPER = results[1]
print_summarized_results(MATH_STATEYEAR__NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9183
Estimator:               Absorbing LS   Adj. R-squared:                     0.9007
No. Observations:               24734   F-statistic:                        294.08
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:23   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0323
                                        Varaibles Absorbed:                 4368.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
virtualper    -9.2024     0.9230    -9.9706     0.0000     -11.011     -7.3935
hybridper     -7.246

#### ELA

In [79]:
results = run_state_year_regression('elapass', 'totaltested')

In [80]:
ELA_STATEYEAR_NOPER = results[0]
print_summarized_results(ELA_STATEYEAR_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9393
Estimator:               Absorbing LS   Adj. R-squared:                     0.9260
No. Observations:               18554   F-statistic:                        177.92
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:23   Distribution:                      chi2(5)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0556
                                        Varaibles Absorbed:                 3338.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode    -0.5433     0.4986    -1.0898     0.2758     -1.5205      0.4338
black         -0.542

In [81]:
ELA_STATEYEAR__PER = results[1]
print_summarized_results(ELA_STATEYEAR__PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9393
Estimator:               Absorbing LS   Adj. R-squared:                     0.9260
No. Observations:               18554   F-statistic:                        180.84
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:23   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0559
                                        Varaibles Absorbed:                 3338.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
virtualper    -0.3886     0.5036    -0.7717     0.4403     -1.3756      0.5984
hybridper      1.369

#### DROPOUT

In [82]:
results = run_state_year_regression('dropout', 'totalenrolled')

In [83]:
DROPOUT_STATEYEAR_NOPER = results[0]
print_summarized_results(DROPOUT_STATEYEAR_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9025
Estimator:               Absorbing LS   Adj. R-squared:                     0.8697
No. Observations:               15568   F-statistic:                        107.39
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:24   Distribution:                      chi2(5)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0101
                                        Varaibles Absorbed:                 3910.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode    -2.0250     0.2530    -8.0032     0.0000     -2.5209     -1.5291
black          0.042

In [84]:
DROPOUT_STATEYEAR_PER = results[1]
print_summarized_results(DROPOUT_STATEYEAR_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9025
Estimator:               Absorbing LS   Adj. R-squared:                     0.8697
No. Observations:               15568   F-statistic:                        109.85
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:24   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0104
                                        Varaibles Absorbed:                 3910.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
virtualper    -1.9840     0.2417    -8.2098     0.0000     -2.4577     -1.5104
hybridper     -0.630

# School FE + District-Year Interaction

### School FE + District-Year Regression Function

In [85]:
def run_district_year_regression(outcome_var, weight):
    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:]
    data['district_year'] = data['districtcode'] + data['year'].astype('str')

    effect = 'district_year'
    outcome_var = outcome_var
    exog_vars = ['schoolmode', 'black', 'hispanic', 'lowincome', 'white']

    absorb = data[['schoolcode', 'year', effect]].astype('category')
    model = AbsorbingLS(dependent=data[outcome_var], exog=data[exog_vars], absorb=absorb, weights=data[weight])
    panel_results_no_per = model.fit(cov_type='clustered', clusters=data['schoolcode'])
    
    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:]
    data['district_year'] = data['districtcode'] + data['year'].astype('str')

    effect = 'district_year'
    outcome_var = outcome_var
    exog_vars = ['virtualper', 'hybridper','black', 'hispanic', 'lowincome', 'white']

    absorb = data[['schoolcode', 'year', effect]].astype('category')
    model = AbsorbingLS(dependent=data[outcome_var], exog=data[exog_vars], absorb=absorb, weights=data[weight])
    panel_results_per = model.fit(cov_type='clustered', clusters=data['schoolcode'])

    return [panel_results_no_per, panel_results_per]

#### MATH

In [86]:
results = run_district_year_regression('mathpass', 'totaltested')

In [87]:
MATH_DISTRICTYEAR__NOPER = results[0]
print_summarized_results(MATH_DISTRICTYEAR__NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9613
Estimator:               Absorbing LS   Adj. R-squared:                     0.9066
No. Observations:               24734   F-statistic:                        229.46
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:24   Distribution:                      chi2(5)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0248
                                        Varaibles Absorbed:              1.449e+04
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode     2.7873     1.8619     1.4970     0.1344     -0.8619      6.4366
black         -0.493

In [88]:
MATH_DISTRICTYEAR__NOPER = results[1]
print_summarized_results(MATH_DISTRICTYEAR__NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9613
Estimator:               Absorbing LS   Adj. R-squared:                     0.9066
No. Observations:               24734   F-statistic:                        230.33
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:24   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0251
                                        Varaibles Absorbed:              1.449e+04
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
virtualper     3.1201     1.8827     1.6572     0.0975     -0.5700      6.8101
hybridper      7.240

#### ELA

In [89]:
results = run_district_year_regression('elapass', 'totaltested')

In [90]:
ELA_DISTRICTYEAR_NOPER = results[0]
print_summarized_results(ELA_DISTRICTYEAR_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9748
Estimator:               Absorbing LS   Adj. R-squared:                     0.9085
No. Observations:               18554   F-statistic:                        108.73
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:25   Distribution:                      chi2(5)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0385
                                        Varaibles Absorbed:              1.344e+04
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode     1.5707     1.6474     0.9534     0.3404     -1.6582      4.7996
black         -0.378

In [91]:
ELA_DISTRICTYEAR__PER = results[1]
print_summarized_results(ELA_DISTRICTYEAR__PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9748
Estimator:               Absorbing LS   Adj. R-squared:                     0.9086
No. Observations:               18554   F-statistic:                        114.25
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:25   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0391
                                        Varaibles Absorbed:              1.344e+04
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
virtualper     1.9828     1.6608     1.1939     0.2325     -1.2723      5.2379
hybridper      7.054

#### DROPOUT

In [92]:
results = run_district_year_regression('dropout', 'totalenrolled')

In [93]:
DROPOUT_DISTRICTYEAR_NOPER = results[0]
print_summarized_results(DROPOUT_DISTRICTYEAR_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9380
Estimator:               Absorbing LS   Adj. R-squared:                     0.7555
No. Observations:               15568   F-statistic:                        24.045
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0002
Time:                        09:26:25   Distribution:                      chi2(5)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0046
                                        Varaibles Absorbed:              1.161e+04
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode    -4.1376     1.0190    -4.0606     0.0000     -6.1347     -2.1405
black          0.012

In [94]:
DROPOUT_DISTRICTYEAR_PER = results[1]
print_summarized_results(DROPOUT_DISTRICTYEAR_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9381
Estimator:               Absorbing LS   Adj. R-squared:                     0.7561
No. Observations:               15568   F-statistic:                        27.601
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0001
Time:                        09:26:25   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0071
                                        Varaibles Absorbed:              1.161e+04
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
virtualper    -2.1116     0.6905    -3.0579     0.0022     -3.4650     -0.7582
hybridper      3.257

# School FE + County-Year Interaction

### School FE + County-Year Regression Function

In [95]:
def run_county_year_regression(outcome_var, weight):
    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:]
    data['county_year'] = data['countycode'] + data['year'].astype('str')

    effect = 'county_year'
    outcome_var = outcome_var
    exog_vars = ['schoolmode', 'black', 'hispanic', 'lowincome', 'white']

    absorb = data[['schoolcode', 'year', effect]].astype('category')
    model = AbsorbingLS(dependent=data[outcome_var], exog=data[exog_vars], absorb=absorb, weights=data[weight])
    panel_results_no_per = model.fit(cov_type='clustered', clusters=data['schoolcode'])
    
    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:]
    data['county_year'] = data['countycode'] + data['year'].astype('str')

    effect = 'county_year'
    outcome_var = outcome_var
    exog_vars = ['virtualper', 'hybridper','black', 'hispanic', 'lowincome', 'white']

    absorb = data[['schoolcode', 'year', effect]].astype('category')
    model = AbsorbingLS(dependent=data[outcome_var], exog=data[exog_vars], absorb=absorb, weights=data[weight])
    panel_results_per = model.fit(cov_type='clustered', clusters=data['schoolcode'])

    return [panel_results_no_per, panel_results_per]

#### MATH

In [96]:
results = run_county_year_regression('mathpass', 'totaltested')

In [97]:
MATH_COUNTYYEAR__NOPER = results[0]
print_summarized_results(MATH_COUNTYYEAR__NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9359
Estimator:               Absorbing LS   Adj. R-squared:                     0.9127
No. Observations:               24734   F-statistic:                        196.88
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:26   Distribution:                      chi2(5)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0197
                                        Varaibles Absorbed:                 6558.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode    -2.1969     1.0728    -2.0478     0.0406     -4.2996     -0.0942
black         -0.468

In [98]:
MATH_COUNTYYEAR__NOPER = results[1]
print_summarized_results(MATH_COUNTYYEAR__NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9359
Estimator:               Absorbing LS   Adj. R-squared:                     0.9127
No. Observations:               24734   F-statistic:                        197.02
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:26   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0197
                                        Varaibles Absorbed:                 6558.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
virtualper    -2.1947     1.1020    -1.9916     0.0464     -4.3545     -0.0348
hybridper     -1.146

#### ELA

In [99]:
results = run_county_year_regression('elapass', 'totaltested')

In [100]:
ELA_COUNTYYEAR_NOPER = results[0]
print_summarized_results(ELA_COUNTYYEAR_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9485
Estimator:               Absorbing LS   Adj. R-squared:                     0.9266
No. Observations:               18554   F-statistic:                        157.06
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:26   Distribution:                      chi2(5)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0503
                                        Varaibles Absorbed:                 5528.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode    -0.4574     0.6746    -0.6781     0.4977     -1.7797      0.8648
black         -0.553

In [101]:
ELA_COUNTYYEAR__PER = results[1]
print_summarized_results(ELA_COUNTYYEAR__PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9485
Estimator:               Absorbing LS   Adj. R-squared:                     0.9267
No. Observations:               18554   F-statistic:                        162.04
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:27   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0507
                                        Varaibles Absorbed:                 5528.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
virtualper    -0.1787     0.6855    -0.2608     0.7943     -1.5223      1.1648
hybridper      1.937

#### DROPOUT

In [102]:
results = run_county_year_regression('dropout', 'totalenrolled')

In [103]:
DROPOUT_COUNTYYEAR_NOPER = results[0]
print_summarized_results(DROPOUT_COUNTYYEAR_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9141
Estimator:               Absorbing LS   Adj. R-squared:                     0.8620
No. Observations:               15568   F-statistic:                        27.283
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0001
Time:                        09:26:27   Distribution:                      chi2(5)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0048
                                        Varaibles Absorbed:                 5874.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode    -1.1138     0.3085    -3.6099     0.0003     -1.7185     -0.5091
black          0.029

In [104]:
DROPOUT_COUNTYYEAR_PER = results[1]
print_summarized_results(DROPOUT_COUNTYYEAR_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9141
Estimator:               Absorbing LS   Adj. R-squared:                     0.8620
No. Observations:               15568   F-statistic:                        27.066
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0001
Time:                        09:26:27   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0049
                                        Varaibles Absorbed:                 5874.0
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
virtualper    -1.1168     0.3110    -3.5915     0.0003     -1.7263     -0.5073
hybridper     -0.349

# School FE + District-Year Interaction + Race Interactions

### School FE + District-Year Interaction + Interactions Regression Function

In [105]:
def run_district_interaction_year_regression(outcome_var, weight, interaction):
    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:]
    data['district_year'] = data['districtcode'] + data['year'].astype('str')
    data[f'{interaction}_int'] = data[interaction] * data['schoolmode']

    effect = 'district_year'
    outcome_var = outcome_var
    exog_vars = ['schoolmode', 'black', 'hispanic', 'lowincome', 'white', f'{interaction}_int']

    absorb = data[['schoolcode', 'year', effect]].astype('category')
    model = AbsorbingLS(dependent=data[outcome_var], exog=data[exog_vars], absorb=absorb, weights=data[weight])
    panel_results_no_per = model.fit(cov_type='clustered', clusters=data['schoolcode'])
    
    data = pd.read_csv(f'../final_data_all_state/final_data_all_state_{outcome_var}.csv').iloc[:, 1:]
    data['district_year'] = data['districtcode'] + data['year'].astype('str')
    data[f'{interaction}_virtual_per'] = data[interaction] * data['virtualper']
    data[f'{interaction}_hybrid_per'] = data[interaction] * data['hybridper']

    effect = 'district_year'
    outcome_var = outcome_var
    exog_vars = ['virtualper', 'hybridper','black', 'hispanic', 'lowincome', 'white', f'{interaction}_virtual_per', f'{interaction}_hybrid_per']

    absorb = data[['schoolcode', 'year', effect]].astype('category')
    model = AbsorbingLS(dependent=data[outcome_var], exog=data[exog_vars], absorb=absorb, weights=data[weight])
    panel_results_per = model.fit(cov_type='clustered', clusters=data['schoolcode'])

    return [panel_results_no_per, panel_results_per]

#### MATH - BLACK

In [106]:
results = run_district_interaction_year_regression('mathpass', 'totaltested', 'black')

In [107]:
MATH_DISTRICTYEAR_BLACK_NOPER = results[0]
print_summarized_results(MATH_DISTRICTYEAR_BLACK_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9613
Estimator:               Absorbing LS   Adj. R-squared:                     0.9066
No. Observations:               24734   F-statistic:                        229.37
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:28   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0249
                                        Varaibles Absorbed:              1.449e+04
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode     2.5848     1.8936     1.3650     0.1722     -1.1265      6.2961
black         -0.493

In [108]:
MATH_DISTRICTYEAR_BLACK_PER = results[1]
print_summarized_results(MATH_DISTRICTYEAR_BLACK_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9613
Estimator:               Absorbing LS   Adj. R-squared:                     0.9066
No. Observations:               24734   F-statistic:                        236.73
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:28   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0253
                                        Varaibles Absorbed:              1.449e+04
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
virtualper            3.4416     1.9122     1.7998     0.0719     -0.3062     

#### ELA - BLACK

In [109]:
results = run_district_interaction_year_regression('elapass', 'totaltested', 'black')

In [110]:
ELA_DISTRICTYEAR_BLACK_NOPER = results[0]
print_summarized_results(ELA_DISTRICTYEAR_BLACK_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9748
Estimator:               Absorbing LS   Adj. R-squared:                     0.9085
No. Observations:               18554   F-statistic:                        109.75
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:28   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0385
                                        Varaibles Absorbed:              1.344e+04
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode     1.3697     1.7184     0.7971     0.4254     -1.9982      4.7377
black         -0.378

In [111]:
ELA_DISTRICTYEAR_BLACK_PER = results[1]
print_summarized_results(ELA_DISTRICTYEAR_BLACK_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9748
Estimator:               Absorbing LS   Adj. R-squared:                     0.9085
No. Observations:               18554   F-statistic:                        116.44
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:29   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0392
                                        Varaibles Absorbed:              1.344e+04
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
virtualper            2.0389     1.7866     1.1412     0.2538     -1.4628     

#### DROPOUT - BLACK

In [112]:
results = run_district_interaction_year_regression('dropout', 'totalenrolled', 'black')

In [113]:
DROPOUT_DISTRICTYEAR_BLACK_NOPER = results[0]
print_summarized_results(DROPOUT_DISTRICTYEAR_BLACK_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9381
Estimator:               Absorbing LS   Adj. R-squared:                     0.7559
No. Observations:               15568   F-statistic:                        29.656
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:29   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0064
                                        Varaibles Absorbed:              1.161e+04
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
schoolmode    -3.7647     0.9940    -3.7876     0.0002     -5.7129     -1.8166
black          0.013

In [114]:
DROPOUT_DISTRICTYEAR_BLACK_PER = results[1]
print_summarized_results(DROPOUT_DISTRICTYEAR_BLACK_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9383
Estimator:               Absorbing LS   Adj. R-squared:                     0.7565
No. Observations:               15568   F-statistic:                        31.007
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0001
Time:                        09:26:29   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0091
                                        Varaibles Absorbed:              1.161e+04
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
virtualper           -1.6556     0.6751    -2.4522     0.0142     -2.9788     

#### ELA - HISPANIC

In [115]:
results = run_district_interaction_year_regression('elapass', 'totaltested', 'hispanic')

In [116]:
ELA_DISTRICTYEAR_HISPANIC_NOPER = results[0]
print_summarized_results(ELA_DISTRICTYEAR_HISPANIC_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9748
Estimator:               Absorbing LS   Adj. R-squared:                     0.9086
No. Observations:               18554   F-statistic:                        119.42
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:29   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0391
                                        Varaibles Absorbed:              1.344e+04
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
schoolmode       3.1747     1.7272     1.8380     0.0661     -0.2106      6.5600
black       

In [117]:
ELA_DISTRICTYEAR_HISPANIC_PER = results[1]
print_summarized_results(ELA_DISTRICTYEAR_HISPANIC_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9748
Estimator:               Absorbing LS   Adj. R-squared:                     0.9086
No. Observations:               18554   F-statistic:                        124.93
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:30   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0397
                                        Varaibles Absorbed:              1.344e+04
                                  Parameter Estimates                                   
                      Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------------
virtualper               3.7665     1.8058     2.0858     0.0370     

#### DROPOUT - BLACK

In [118]:
results = run_district_interaction_year_regression('dropout', 'totalenrolled', 'hispanic')

In [119]:
DROPOUT_DISTRICTYEAR_HISPANIC_NOPER = results[0]
print_summarized_results(DROPOUT_DISTRICTYEAR_HISPANIC_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9380
Estimator:               Absorbing LS   Adj. R-squared:                     0.7555
No. Observations:               15568   F-statistic:                        26.050
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0002
Time:                        09:26:30   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0046
                                        Varaibles Absorbed:              1.161e+04
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
schoolmode      -4.0564     1.1844    -3.4249     0.0006     -6.3778     -1.7350
black       

In [120]:
DROPOUT_DISTRICTYEAR_HISPANIC_PER = results[1]
print_summarized_results(DROPOUT_DISTRICTYEAR_HISPANIC_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9384
Estimator:               Absorbing LS   Adj. R-squared:                     0.7571
No. Observations:               15568   F-statistic:                        54.253
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:30   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0118
                                        Varaibles Absorbed:              1.161e+04
                                  Parameter Estimates                                   
                      Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------------
virtualper              -3.9606     1.1226    -3.5280     0.0004     

# School FE + District-Year Interaction + Other Interactions

### School FE + District-Year Interaction + Income Interactions Regression Function

#### MATH - LOWINCOME

In [121]:
results = run_district_interaction_year_regression('mathpass', 'totaltested', 'lowincome')

In [122]:
MATH_DISTRICTYEAR_LOWINCOME_NOPER = results[0]
print_summarized_results(MATH_DISTRICTYEAR_LOWINCOME_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9613
Estimator:               Absorbing LS   Adj. R-squared:                     0.9066
No. Observations:               24734   F-statistic:                        229.46
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:31   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0249
                                        Varaibles Absorbed:              1.449e+04
                               Parameter Estimates                               
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
schoolmode        2.1836     2.4528     0.8902     0.3733     -2.6239      6.9910
black   

In [123]:
MATH_DISTRICTYEAR_LOWINCOME_PER = results[1]
print_summarized_results(MATH_DISTRICTYEAR_LOWINCOME_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9613
Estimator:               Absorbing LS   Adj. R-squared:                     0.9066
No. Observations:               24734   F-statistic:                        230.64
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:31   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0251
                                        Varaibles Absorbed:              1.449e+04
                                   Parameter Estimates                                   
                       Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------------
virtualper                2.5224     2.6661     0.9461     0.3441 

#### ELA - LOWINCOME

In [124]:
results = run_district_interaction_year_regression('elapass', 'totaltested', 'lowincome')

In [125]:
ELA_DISTRICTYEAR_LOWINCOME_NOPER = results[0]
print_summarized_results(ELA_DISTRICTYEAR_LOWINCOME_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9748
Estimator:               Absorbing LS   Adj. R-squared:                     0.9085
No. Observations:               18554   F-statistic:                        111.47
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:31   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0385
                                        Varaibles Absorbed:              1.344e+04
                               Parameter Estimates                               
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
schoolmode        1.9647     1.9627     1.0010     0.3168     -1.8821      5.8115
black   

In [126]:
ELA_DISTRICTYEAR_LOWINCOME_PER = results[1]
print_summarized_results(ELA_DISTRICTYEAR_LOWINCOME_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9748
Estimator:               Absorbing LS   Adj. R-squared:                     0.9085
No. Observations:               18554   F-statistic:                        118.21
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:31   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0393
                                        Varaibles Absorbed:              1.344e+04
                                   Parameter Estimates                                   
                       Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------------
virtualper                2.7411     2.0505     1.3368     0.1813 

#### DROPOUT - LOWINCOME

In [127]:
results = run_district_interaction_year_regression('dropout', 'totalenrolled', 'lowincome')

In [128]:
DROPOUT_DISTRICTYEAR_LOWINCOME_NOPER = results[0]
print_summarized_results(DROPOUT_DISTRICTYEAR_LOWINCOME_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9383
Estimator:               Absorbing LS   Adj. R-squared:                     0.7567
No. Observations:               15568   F-statistic:                        48.116
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:32   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0095
                                        Varaibles Absorbed:              1.161e+04
                               Parameter Estimates                               
               Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------
schoolmode       -1.1642     1.1037    -1.0549     0.2915     -3.3274      0.9989
black   

In [129]:
DROPOUT_DISTRICTYEAR_LOWINCOME_PER = results[1]
print_summarized_results(DROPOUT_DISTRICTYEAR_LOWINCOME_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9384
Estimator:               Absorbing LS   Adj. R-squared:                     0.7571
No. Observations:               15568   F-statistic:                        69.137
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:32   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0119
                                        Varaibles Absorbed:              1.161e+04
                                   Parameter Estimates                                   
                       Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------------
virtualper               -0.3928     1.1003    -0.3570     0.7211 

### School FE + District-Year Interaction + Charter Interactions Regression Function

#### MATH - CHARTER

In [130]:
results = run_district_interaction_year_regression('mathpass', 'totaltested', 'charter')

In [131]:
MATH_DISTRICTYEAR_CHARTER_NOPER = results[0]
print_summarized_results(MATH_DISTRICTYEAR_CHARTER_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9614
Estimator:               Absorbing LS   Adj. R-squared:                     0.9066
No. Observations:               24734   F-statistic:                        233.40
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:32   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0256
                                        Varaibles Absorbed:              1.449e+04
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
schoolmode      3.5313     1.9007     1.8579     0.0632     -0.1941      7.2566
black          -

In [132]:
MATH_DISTRICTYEAR_CHARTER_PER = results[1]
print_summarized_results(MATH_DISTRICTYEAR_CHARTER_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:               mathpass   R-squared:                          0.9614
Estimator:               Absorbing LS   Adj. R-squared:                     0.9067
No. Observations:               24734   F-statistic:                        236.92
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:33   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0267
                                        Varaibles Absorbed:              1.449e+04
                                  Parameter Estimates                                  
                     Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------
virtualper              4.4757     1.9658     2.2768     0.0228      0.6

#### ELA - CHARTER

In [133]:
results = run_district_interaction_year_regression('elapass', 'totaltested', 'charter')

In [134]:
ELA_DISTRICTYEAR_CHARTER_NOPER = results[0]
print_summarized_results(ELA_DISTRICTYEAR_CHARTER_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9748
Estimator:               Absorbing LS   Adj. R-squared:                     0.9087
No. Observations:               18554   F-statistic:                        113.87
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:33   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0402
                                        Varaibles Absorbed:              1.344e+04
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
schoolmode      2.3729     1.7341     1.3684     0.1712     -1.0258      5.7716
black          -

In [135]:
ELA_DISTRICTYEAR_CHARTER_PER = results[1]
print_summarized_results(ELA_DISTRICTYEAR_CHARTER_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                elapass   R-squared:                          0.9749
Estimator:               Absorbing LS   Adj. R-squared:                     0.9087
No. Observations:               18554   F-statistic:                        119.29
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0000
Time:                        09:26:33   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0411
                                        Varaibles Absorbed:              1.344e+04
                                  Parameter Estimates                                  
                     Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------
virtualper              3.0457     1.7726     1.7182     0.0858     -0.4

#### DROPOUT - CHARTER

In [136]:
results = run_district_interaction_year_regression('dropout', 'totalenrolled', 'charter')

In [137]:
DROPOUT_DISTRICTYEAR_CHARTER_NOPER = results[0]
print_summarized_results(DROPOUT_DISTRICTYEAR_CHARTER_NOPER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9380
Estimator:               Absorbing LS   Adj. R-squared:                     0.7555
No. Observations:               15568   F-statistic:                        24.312
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0005
Time:                        09:26:34   Distribution:                      chi2(6)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0046
                                        Varaibles Absorbed:              1.161e+04
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
schoolmode     -4.3228     1.3344    -3.2395     0.0012     -6.9381     -1.7074
black           

In [138]:
DROPOUT_DISTRICTYEAR_CHARTER_PER = results[1]
print_summarized_results(DROPOUT_DISTRICTYEAR_CHARTER_PER)

                         Absorbing LS Estimation Summary                          
Dep. Variable:                dropout   R-squared:                          0.9382
Estimator:               Absorbing LS   Adj. R-squared:                     0.7560
No. Observations:               15568   F-statistic:                        29.046
Date:                Sat, Sep 23 2023   P-value (F-stat):                   0.0003
Time:                        09:26:34   Distribution:                      chi2(8)
Cov. Estimator:             clustered   R-squared (No Effects):             0.0074
                                        Varaibles Absorbed:              1.161e+04
                                  Parameter Estimates                                  
                     Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------
virtualper             -2.6332     1.3592    -1.9373     0.0527     -5.2